In [1]:
"""
Try stuff for Georgia
"""

'\nTry stuff for Georgia\n'

In [23]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import numpy as np
import matplotlib as mpl

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [4]:
inputpath = '/data/cburgard/FOR_GEORGIA/'

In [5]:
data_1deg = xr.open_dataset(inputpath + 'NEMO_1degree/n42openc_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
data_025deg = xr.open_dataset(inputpath + 'NEMO_025degree/MJ2023_eORCA025.L121-OPM026_y1989.30y_gridT_timmean.nc').squeeze().drop('time_counter')

In [7]:
mask_ocean = data_1deg['so'] > 0
#mask_ocean = data_025deg['vosaline'] > 0

In [8]:
mask_ocean.isel(deptht=50).plot()

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


In [11]:
# masque là où il y a de l'océan
mask_ocean = data_1deg['so'] > 0
#mask_ocean = data_025deg['vosaline'] > 0

# exemple pour un point sous un ice shelf
col_test = mask_ocean.sel(x=232,y=24)
#col_test = mask_ocean.sel(x=912,y=167)

# shifter le masque d'un point sur la verticale et faire une différence avec le masque originale 
# si 1 ou -1, changement de non-océan à océan
vert_diff_plus = (col_test - col_test.shift(deptht=1)).isel(deptht=range(1,len(col_test.deptht)))
vert_diff_minus = (col_test - col_test.shift(deptht=-1)).isel(deptht=range(1,len(col_test.deptht)))

# dernier point d'océan avant la glace
ice_depth = col_test.deptht.where(vert_diff_plus == 1, drop=True)
# dernier point d'océan avant le fond
bot_depth = col_test.deptht.where(vert_diff_minus == 1, drop=True)

In [29]:
mask_ocean.isel(deptht=50).plot()

In [34]:
col_test.plot()
plt.axvline(x=ice_depth)
plt.axvline(x=bot_depth)

In [12]:
# pour tous les points
vert_diff_plus_all = (mask_ocean - mask_ocean.shift(deptht=1)).isel(deptht=range(1,len(mask_ocean.deptht)))
vert_diff_minus_all = (mask_ocean - mask_ocean.shift(deptht=-1)).isel(deptht=range(1,len(mask_ocean.deptht)))

In [13]:
# dernier point d'océan avant la glace - somme de toutes les profondeurs où la différence est positive (donc la transition glace-océan)
ice_depth_all = (mask_ocean.deptht * vert_diff_plus_all).where(vert_diff_plus_all > 0).sum('deptht').astype('float')
# dernier point d'océan avant le fond - somme de toutes les profondeurs où la différence est positive (donc la transition océan-fond)
bot_depth_all = (mask_ocean.deptht * vert_diff_minus_all).where(vert_diff_minus_all > 0).sum('deptht').astype('float')

In [73]:
ice_depth_all = ice_depth_all.where(ice_depth_all > 0,0)

In [17]:
lon = ice_depth_all.nav_lon
lat = ice_depth_all.nav_lat

In [33]:
# Function z_masked_overlap() to prevent the line wrapping in contour plots
# The function z_masked_overlap was taken from
# https://github.com/SciTools/cartopy/issues/1421 from htonchia.

def z_masked_overlap(axe, X, Y, Z, source_projection=None):
    """
    for data in projection axe.projection
    find and mask the overlaps (more 1/2 the axe.projection range)

    X, Y either the coordinates in axe.projection or longitudes latitudes
    Z the data
    operation one of 'pcorlor', 'pcolormesh', 'countour', 'countourf'

    if source_projection is a geodetic CRS data is in geodetic coordinates
    and should first be projected in axe.projection

    X, Y are 2D same dimension as Z for contour and contourf
    same dimension as Z or with an extra row and column for pcolor
    and pcolormesh

    return ptx, pty, Z
    """
    if not hasattr(axe, 'projection'):
        return X, Y, Z

    if not isinstance(axe.projection, ccrs.Projection):
        return X, Y, Z

    if len(X.shape) != 2 or len(Y.shape) != 2:
        return X, Y, Z

    if (source_projection is not None and
        isinstance(source_projection, ccrs.Geodetic)):
        transformed_pts = axe.projection.transform_points(
            source_projection, X, Y)
        ptx, pty = transformed_pts[..., 0], transformed_pts[..., 1]
    else:
        ptx, pty = X, Y

    with np.errstate(invalid='ignore'):
        # diagonals have one less row and one less columns
        diagonal0_lengths = np.hypot(
            ptx[1:, 1:] - ptx[:-1, :-1],
            pty[1:, 1:] - pty[:-1, :-1]
        )

        diagonal1_lengths = np.hypot(
            ptx[1:, :-1] - ptx[:-1, 1:],
            pty[1:, :-1] - pty[:-1, 1:]
        )

        to_mask = (
            (diagonal0_lengths > (
                abs(axe.projection.x_limits[1]
                    - axe.projection.x_limits[0])) / 2) |
            np.isnan(diagonal0_lengths) |
            (diagonal1_lengths > (
                abs(axe.projection.x_limits[1]
                    - axe.projection.x_limits[0])) / 2) |
            np.isnan(diagonal1_lengths)
        )

        # TODO check if we need to do something about surrounding vertices
        # add one extra colum and row for contour and contourf

        if (to_mask.shape[0] == Z.shape[0] - 1 and
                to_mask.shape[1] == Z.shape[1] - 1):
            to_mask_extended = np.zeros(Z.shape, dtype=bool)
            to_mask_extended[:-1, :-1] = to_mask
            to_mask_extended[-1, :] = to_mask_extended[-2, :]
            to_mask_extended[:, -1] = to_mask_extended[:, -2]
            to_mask = to_mask_extended

        if np.any(to_mask):
            Z_mask = getattr(Z, 'mask', None)
            to_mask = to_mask if Z_mask is None else to_mask | Z_mask
            Z = np.ma.masked_where(to_mask, Z)

        return ptx, pty, Z

In [63]:
# ton array de profondeur
bathymetry = bot_depth_all
# les niveaux que tu veux avoir
levels = np.arange(0,5000,100)


f = plt.figure(figsize=(8.25*1.5,8.25/1.5))

proj = ccrs.SouthPolarStereo(central_longitude=0)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)        

ax1 = plt.subplot(1, 1, 1, projection=proj)

#-- Compute the corrected data to prevent line wrapping. Use the dataset
#-- coordinates and variable.
X, Y, maskedZ = z_masked_overlap(ax1,
                                 lon.data,
                                 lat.data,
                                 bathymetry,
                                 source_projection=ccrs.Geodetic())

plotb = ax1.contour(X, Y, maskedZ,
                     levels=levels,
                     linewidths=1.0,
                    colors='black')

# ça ne marche pas le truc des labels mais bon tant pis
ax1.clabel(plotb, plotb.levels[::5], inline=True, fontsize=8, fmt='%1.0f')

#ax1.coastlines(resolution='110m', linewidth=0.5)

ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

    

In [58]:
plotb.levels

array([   0.,  250.,  500.,  750., 1000., 1250., 1500., 1750., 2000.,
       2250., 2500., 2750., 3000., 3250., 3500., 3750., 4000., 4250.,
       4500., 4750.])

In [41]:
((mask_ocean.deptht * vert_diff_plus_all) > 1)

<xarray.DataArray (deptht: 120, y: 1207, x: 1442)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
...
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])
Coordinates:
  * deptht   (deptht) float32 1.526 2.638 3.84 ... 5.494e+03 5.698e+03 5.902e+03
    nav_lon  (y, x) float32 -1.0 -1.0 -1.0 -1.0 -1.0 ... -1.0 -1.0 -1.0 -1.0
    nav_lat  (y, x) float32 -1.0 -1.0 -1.0 -1.0 -1.0 ... -1.0 -1.0 -1.0 -1.0
Dimensions without coordinates: y, x

In [82]:
data_1deg['so'].where((ice_depth_all > 0) & mask_ocean).max('deptht').plot()

In [71]:
data_1deg['so'].where(data_1deg['so'].(deptht=ice_depth_all)

KeyError: "not all values found in index 'deptht'"

In [70]:
bot_depth_all.plot()

In [53]:
col_test.plot()
plt.axvline(x=ice_depth)
plt.axvline(x=bot_depth)

In [39]:
bot_depth

<xarray.DataArray 'deptht' (deptht: 0)>
array([], dtype=float32)
Coordinates:
    nav_lat  float32 -81.67
    nav_lon  float32 -56.57
  * deptht   (deptht) float32 
Attributes:
    name:       deptht
    long_name:  Vertical T levels
    units:      m
    positive:   down
    bounds:     deptht_bounds